In [1]:
#First we import all required modules
import numpy as np
import itertools
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
from scipy.integrate import quad
import io, json

init_notebook_mode(connected=True)

#Then we define any constants to be used in later calculations
a = 0.35;
shape = "circle" 
perm = 4*np.pi*10e-7 
current = 2.; 
P = np.array([0,0,1]);
phi_scal = 0
B = np.empty((0, 3))


In [2]:
#

def calc_this(shape,phi_scal,P,a):
    if shape == "circle":
        dI= np.array([-a*np.sin(phi_scal),a*np.cos(phi_scal),0])
        r_vec = np.array([P[0]-a*np.cos(phi_scal),P[1]-a*np.sin(phi_scal),P[2]])
        
    return dI,r_vec 

def Biot_equation (current,perm,dI,r_vec):
    xprod = np.cross(dI,r_vec)
    dB = (current/(4*np.pi*(np.linalg.norm(r_vec)**3)))*xprod #should multiply current by perm 
    return dB

def calc_integrand(phi):
    a = 0.5;
    dI= np.array([-a*np.sin(phi),a*np.cos(phi),0])
    rvec = np.array([P[0]-a*np.cos(phi),P[1]-a*np.sin(phi),P[2]])
    result =(current/(4*np.pi*(np.linalg.norm(r_vec)**3)))* np.cross(dI,rvec)
    return result[i]

dI,r_vec = calc_this("circle",phi_scal,P,a)
dB= 2*Biot_equation(current,perm,dI,r_vec) #value of 18 is heuristic for diagram 

for j in range(3):
    i=j
    zz = quad(calc_integrand,0,2*np.pi)[0]
    B = np.append(B,zz,axis=None) #value of 2.5 is heuristic for diagram 




In [3]:
data = {}

uu = np.linspace(0, 2 * np.pi, 50)
xc = a*np.cos(uu); 
yc = a*np.sin(uu); 
zc = np.zeros_like(xc); 


for i in range(0,5):
    data[i]={}
    B = [0,0,0]
    P= [0,0,1]
    for j in range(0,10):
        phi_scal = 2*np.pi*j/10
        dI,r_vec = calc_this("circle",phi_scal,P,a)
        dB= 2*Biot_equation(current,perm,dI,r_vec)
        for n in range(3):
            B[n] += dB[n]
        data[i][j]={}
        data[i][j][0]={"name": "dI","x":[a*np.cos(phi_scal),dI[0]+a*np.cos(phi_scal)],"y":[a*np.sin(phi_scal),dI[1]+a*np.sin(phi_scal)],"z":[0,dI[2]]}
        data[i][j][1]={"name": "dB", "x":[P[0],P[0]+dB[0]],"y":[P[1],P[1]+dB[1]],"z":[P[2],P[2]+dB[2]]}
        data[i][j][2]={"name": "R","x":[a*np.cos(phi_scal),r_vec[0]+a*np.cos(phi_scal)],"y":[a*np.sin(phi_scal),r_vec[1]+a*np.sin(phi_scal)],"z":[0,r_vec[2]]}
        data[i][j][3]={"name": "B","x":[P[0],P[0]+B[0]],"y":[P[1],P[1]+B[1]],"z":[P[2],P[2]+B[2]]}
        data[i][j][4]={"name": "Current Loop","x":xc,"y":yc,"z":zc}
 
print(data[0][0][0]["x"])
result=[]
m=4
for i in range(0,10):
    result.append(go.Scatter3d(x=data[0][i][m]["x"],y=data[0][i][m]["y"],z=data[0][i][m]["z"]))

fig = dict(data=result)

iplot(fig)
print(data)
dump = json.dumps(data)
with open('JSONData.json', 'w') as f:
     json.dump(dump, f)

[0.35, 0.35]


{0: {0: {0: {'name': 'dI', 'x': [0.35, 0.35], 'y': [0.0, 0.35], 'z': [0, 0.0]}, 1: {'name': 'dB', 'x': [0, 0.09367822369351479], 'y': [0, 0.0], 'z': [1, 1.0327873782927302]}, 2: {'name': 'R', 'x': [0.35, 0.0], 'y': [0.0, 0.0], 'z': [0, 1.0]}, 3: {'name': 'B', 'x': [0, 0.09367822369351479], 'y': [0, 0.0], 'z': [1, 1.0327873782927302]}, 4: {'name': 'Current Loop', 'x': array([ 0.35      ,  0.3471265 ,  0.3385532 ,  0.32442087,  0.30496155,
        0.28049477,  0.25142227,  0.21822143,  0.1814374 ,  0.14167417,
        0.09958466,  0.05585996,  0.01121805, -0.03360806, -0.07788233,
       -0.12087777, -0.1618884 , -0.20024083, -0.23530531, -0.26650609,
       -0.29333084, -0.3153391 , -0.33216951, -0.3435457 , -0.34928089,
       -0.34928089, -0.3435457 , -0.33216951, -0.3153391 , -0.29333084,
       -0.26650609, -0.23530531, -0.20024083, -0.1618884 , -0.12087777,
       -0.07788233, -0.03360806,  0.01121805,  0.05585996,  0.09958466,
        0.14167417,  0.1814374 ,  0.21822143,  0.25142

TypeError: Object of type 'ndarray' is not JSON serializable

In [ ]:
"""Plotting script """

Currenttrace = go.Scatter3d(x = [a*np.cos(phi_scal),dI[0]+a*np.cos(phi_scal)], 
                      y = [a*np.sin(phi_scal),dI[1]+a*np.sin(phi_scal)], 
                      z = [0,dI[2]],
                      name = "dI", 
                      line = dict(width = 6,color = 'rgb(250,20,0)'),
                      marker = dict(size=[0,15],color= 'rgb(250,20,0)',symbol= 'diamond',opacity=1)
                      
                      )   


Magtrace = go.Scatter3d(x = [P[0],P[0]+dB[0]], 
                      y = [P[1],P[1]+dB[1]], 
                      z = [P[2],P[2]+dB[2]],
                      name = "dB",
                      line = dict(width = 6, color = 'rgb(50,100,200)'),
                      marker = dict(size=[0,15],color= 'rgb(50,100,200)',symbol= 'diamond',opacity=1)
                     );

Rtrace = go.Scatter3d(x = [a*np.cos(phi_scal),r_vec[0]+a*np.cos(phi_scal)],
                      y = [a*np.sin(phi_scal),r_vec[1]+a*np.sin(phi_scal)],
                      z = [0,r_vec[2]],
                      name = "R", 
                      line = dict(width = 6, color = 'rgb(0,220,0)'),
                      marker = dict(size=[0,15],color= 'rgb(0,220,0)',symbol= 'diamond',opacity=1),
                     );


Btrace = go.Scatter3d(x = [P[0],P[0]+B[0]], 
                      y = [P[1],P[1]+B[1]], 
                      z = [P[2],P[2]+B[2]],
                      name = "B field",
                      line = dict(width = 6, color = 'rgb(255,160,75)'),
                      marker = dict(size=[0,15],color= 'rgb(255,160,122)',symbol= 'diamond',opacity=1)
                     );

Circletrace = go.Scatter3d( x =  xc ,
                      y = yc,
                      z = zc,
                      name = "circle",
                     mode = "markers",
                     marker = dict(symbol="circle",size=2.5,opacity=1,color= 'rgb(0,0,0)'),
                    );






result= [Currenttrace,Magtrace,Rtrace,Btrace,Circletrace]

fig = dict(data=result)

iplot(fig)
